This one looks prosaic and time consuming, rather than elegant, but I'm on leave now, so it doesn't matter.

In [1]:
import re

Rather than rotate each square as we look at it, let's write some parsing functions that take a line of the input, and returns a `dict` mapping input squares to output squares. Assume that the inputs will be correctly in a calling function:

In [2]:
def parse_rule_2(ruleIn_str):
    '''
    Parses a rule for a 2x2 square. Returns a dict
    of rules in the input rule format.
    '''
    
    # Can do a simple pattern match to find the 
    # input and output strings
    [ruleIn_str, ruleOut_str]=[s.strip() for s in ruleIn_str.split('=>')]
    
    # We can use the indexing of a dictionary to 
    # avoid checking for duplicates
    
    # Long and boring way of doing this, but I
    # can't be bothered to do it properly at
    # the moment.
    
    # Parse the input intod characters:
    [a, b, slash, d, c]=list(ruleIn_str)
    
    rulesOut_dict={}
    
    # Add the rotational versions to the dict:
    
    # Rotations are: ab   da   cd   bc
    #                dc   cb   ba   ad
    
    rulesOut_dict['{}{}/{}{}'.format(a, b, d, c)]=ruleOut_str
    rulesOut_dict['{}{}/{}{}'.format(d, a, c, b)]=ruleOut_str
    rulesOut_dict['{}{}/{}{}'.format(c, d, b, a)]=ruleOut_str
    rulesOut_dict['{}{}/{}{}'.format(b, c, a, d)]=ruleOut_str
        
    # Add the flipped versions to the dict:
    
    # Flips are: ad   ba   dc   cb
    #            bc   cd   ab   da

    rulesOut_dict['{}{}/{}{}'.format(a, d, b, c)]=ruleOut_str
    rulesOut_dict['{}{}/{}{}'.format(b, a, c, d)]=ruleOut_str
    rulesOut_dict['{}{}/{}{}'.format(d, c, a, b)]=ruleOut_str
    rulesOut_dict['{}{}/{}{}'.format(c, b, d, a)]=ruleOut_str

    return rulesOut_dict

Do a similar one for 3x3 rules:

In [3]:
def parse_rule_3(ruleIn_str):
    '''
    Parses a rule for a 3x3 square. Returns a dict
    of rules in the input rule format.
    '''
    
    # Can do a simple pattern match to find the 
    # input and output strings
    [ruleIn_str, ruleOut_str]=[s.strip() for s in ruleIn_str.split('=>')]
    
    # We can use the indexing of a dictionary to 
    # avoid checking for duplicates
    
    # Long and boring way of doing this, but I
    # can't be bothered to do it properly at
    # the moment.
    
    # Parse the input into characters:
    [a, b, c, slash1, h, x, d, slash2, g, f, e]=list(ruleIn_str)
    
    rulesOut_dict={}
    
    # Add the rotational versions to the dict:
    
    # Rotations are: abc  gha  efg  cde
    #                hxd  fxb  dxh  bxf
    #                gfe  edc  cba  ahg
    
    rulesOut_dict['{}{}{}/{}{}{}/{}{}{}'.format(a, b, c, h, x, d, g, f, e)]=ruleOut_str
    rulesOut_dict['{}{}{}/{}{}{}/{}{}{}'.format(g, h, a, f, x, b, e, d, c)]=ruleOut_str
    rulesOut_dict['{}{}{}/{}{}{}/{}{}{}'.format(e, f, g, d, x, h, c, b, a)]=ruleOut_str
    rulesOut_dict['{}{}{}/{}{}{}/{}{}{}'.format(c, d, e, b, x, f, a, h, g)]=ruleOut_str
        
    # Add the flipped versions to the dict:
    
    # Flips are: gfe  cba  edc  ahg
    #            hxd  dxh  fxb  bxf
    #            abc  efg  gha  cde

    rulesOut_dict['{}{}{}/{}{}{}/{}{}{}'.format(g, f, e, h, x, d, a, b, c)]=ruleOut_str
    rulesOut_dict['{}{}{}/{}{}{}/{}{}{}'.format(c, b, a, d, x, h, e, f, g)]=ruleOut_str
    rulesOut_dict['{}{}{}/{}{}{}/{}{}{}'.format(e, d, c, f, x, b, g, h, a)]=ruleOut_str
    rulesOut_dict['{}{}{}/{}{}{}/{}{}{}'.format(a, h, g, b, x, f, c, d, e)]=ruleOut_str

    return rulesOut_dict

In [4]:
testInput_str='''
../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#
'''

OK, now we need to parse the pattern book, and return the whole thing as a dictionary. Easiest to use `re` for this, I think. That will mean for a 2x2 rule we want the match:

In [5]:
re.findall('((\.|#){2}/(\.|#){2}\s+\=\>\s+(\.|#){3}/(\.|#){3}/(\.|#){3})', 
           testInput_str)

[('../.# => ##./#../...', '.', '#', '.', '.', '.')]

Note that it's the first element that we want, so to actually parse the input we'd use:

In [6]:
for match_tuple in re.findall('((\.|#){2}/(\.|#){2}\s+\=\>\s+(\.|#){3}/(\.|#){3}/(\.|#){3})', 
                              testInput_str):
    print(match_tuple[0])

../.# => ##./#../...


And we can do something similar for the 3x3 rules:

In [7]:
for match_tuple in re.findall('((\.|#){3}/(\.|#){3}/(\.|#){3}\s+\=\>\s+(\.|#){4}/(\.|#){4}/(\.|#){4}/(\.|#){4})',
                              testInput_str):
    print(match_tuple[0])

.#./..#/### => #..#/..../..../#..#


OK, so the matchers seem to be working. Now we need functions for extracting a subsquare from an input, and inserting a subsquare into an input. We'll describe the input as a list of lists, such as:

`[['.', '#', '.'], ['.', '.', '#'], ['#', '#', '#']]`

as strings are immutable.

In [8]:
def extract_subsquare(topCornerRow_i, topCornerCol_i, size_i, inputSquare_ls):
    '''
    Extract the subsquare whose top corner is at
    (topCornerRow_i, topCornerCol_i) and whose sides are
    length size_i'''
    
    output_str=''
    
    for i in range(topCornerRow_i, topCornerRow_i+size_i):
        for j in range(topCornerCol_i, topCornerCol_i+size_i):
            output_str+=inputSquare_ls[i][j]
        output_str+='/'
    return output_str[:-1]  # Remove trailing slash

        

In [9]:
initialSquare_ls=[['.', '#', '.'], ['.', '.', '#'], ['#', '#', '#']]

assert extract_subsquare(0,0,2,initialSquare_ls)=='.#/..'
assert extract_subsquare(1,0,2,initialSquare_ls)=='../##'
assert extract_subsquare(0,1,2,initialSquare_ls)=='#./.#'
assert extract_subsquare(0,0,3,initialSquare_ls)=='.#./..#/###'


Now a "function" to insert a subsquare into a square. Do it in place; the corruption's got far enough that there's no point in trying to stop it...

In [10]:
def insert_subsquare(topCornerRow_i, topCornerCol_i, input_str, square_ls):
    '''
    Insert a subsquare whose top corner is at
    (topCornerRow_i, topCornerCol_i) and which is defined
    by input_str
    '''
    
    for (i, row_str) in enumerate(input_str.split('/')):
        for (j,c) in enumerate(row_str):
            square_ls[topCornerRow_i+i][topCornerCol_i+j]=c

        

In [11]:
testInputSquare_ls=[[0 for i in range(5)] for j in range(5)]
testInputSquare_ls

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [12]:
insert_subsquare(1, 2, '##./#../...', testInputSquare_ls)
testInputSquare_ls

[[0, 0, 0, 0, 0],
 [0, 0, '#', '#', '.'],
 [0, 0, '#', '.', '.'],
 [0, 0, '.', '.', '.'],
 [0, 0, 0, 0, 0]]

So we should now be able to extract the same string that we input...

In [13]:
testInputSquare_ls
testString_str='##./#../...'

insert_subsquare(1, 2, testString_str, testInputSquare_ls)
assert extract_subsquare(1, 2, 3, testInputSquare_ls)==testString_str

OK, let's bring it all together in the test cases:

First parse the input:

In [14]:
testRules_str='''
../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#
'''

rules_dict={}
for rule_str in re.findall('((\.|#){2}/(\.|#){2}\s+\=\>\s+(\.|#){3}/(\.|#){3}/(\.|#){3})', 
                              testRules_str):
    rules_dict.update(parse_rule_2(rule_str[0]))
for rule_str in re.findall('((\.|#){3}/(\.|#){3}/(\.|#){3}\s+\=\>\s+(\.|#){4}/(\.|#){4}/(\.|#){4}/(\.|#){4})',
                              testInput_str):
    rules_dict.update(parse_rule_3(rule_str[0]))
    
rules_dict
    

{'###/#../.#.': '#..#/..../..../#..#',
 '###/..#/.#.': '#..#/..../..../#..#',
 '##./#.#/#..': '#..#/..../..../#..#',
 '#../#.#/##.': '#..#/..../..../#..#',
 '#./..': '##./#../...',
 '.##/#.#/..#': '#..#/..../..../#..#',
 '.#./#../###': '#..#/..../..../#..#',
 '.#./..#/###': '#..#/..../..../#..#',
 '.#/..': '##./#../...',
 '..#/#.#/.##': '#..#/..../..../#..#',
 '../#.': '##./#../...',
 '../.#': '##./#../...'}

God, this is a lot of work. Let's have a function that takes an input square and the rules, and returns an output square:

In [15]:
def next_state(rulesIn_dict, squareIn_ls):
    '''
    Return the square that's the result of applying
    rulesIn_dict to squareIn_ls
    '''
    
    # I can no longer be bothered to do any error checking
    if not len(squareIn_ls)%2:
        size_i=len(squareIn_ls)//2
        squareOut_ls=[[0 for i in range(3*size_i)] for j in range(3*size_i)]

        for i in range(size_i):
            for j in range(size_i):
                in_str=extract_subsquare(2*i, 2*j, 2, squareIn_ls)
                insert_subsquare(3*i, 3*j, rulesIn_dict[in_str], squareOut_ls)
                
        return squareOut_ls
    
    # If it's not divisible by 2, then it's divisible by 3:
    else:
        size_i=len(squareIn_ls)//3
        squareOut_ls=[[0 for i in range(4*size_i)] for j in range(4*size_i)]

        for i in range(size_i):
            for j in range(size_i):
                in_str=extract_subsquare(3*i, 3*j, 3, squareIn_ls)
                insert_subsquare(4*i, 4*j, rulesIn_dict[in_str], squareOut_ls)
                
        return squareOut_ls
    


In [16]:
next_state(rules_dict, [['.', '#', '.'], ['.', '.', '#'], ['#', '#', '#']])

[['#', '.', '.', '#'],
 ['.', '.', '.', '.'],
 ['.', '.', '.', '.'],
 ['#', '.', '.', '#']]

In [17]:
next_state(rules_dict, _)

[['#', '#', '.', '#', '#', '.'],
 ['#', '.', '.', '#', '.', '.'],
 ['.', '.', '.', '.', '.', '.'],
 ['#', '#', '.', '#', '#', '.'],
 ['#', '.', '.', '#', '.', '.'],
 ['.', '.', '.', '.', '.', '.']]

OK, that seems to be working.

Now let's do it with the puzzle input:

In [18]:
with open('data/day21.txt') as fIn:
    puzzleInput_str=fIn.read()


rules_dict={}
for rule_str in re.findall('((\.|#){2}/(\.|#){2}\s+\=\>\s+(\.|#){3}/(\.|#){3}/(\.|#){3})', 
                              puzzleInput_str):
    rules_dict.update(parse_rule_2(rule_str[0]))
for rule_str in re.findall('((\.|#){3}/(\.|#){3}/(\.|#){3}\s+\=\>\s+(\.|#){4}/(\.|#){4}/(\.|#){4}/(\.|#){4})',
                              puzzleInput_str):
    rules_dict.update(parse_rule_3(rule_str[0]))



Now apply these rules to the input:

In [19]:
puzzleInput_ls=[['.', '#', '.'], ['.', '.', '#'], ['#', '#', '#']]

In [20]:
# Iteration 1
next_state(rules_dict, puzzleInput_ls)

[['.', '#', '.', '#'],
 ['#', '.', '.', '.'],
 ['.', '.', '#', '.'],
 ['#', '.', '#', '#']]

In [21]:
# Iteration 2
next_state(rules_dict, _)

[['#', '.', '.', '.', '#', '#'],
 ['#', '.', '.', '#', '.', '.'],
 ['.', '#', '.', '#', '#', '#'],
 ['.', '#', '#', '.', '#', '.'],
 ['#', '.', '.', '#', '.', '.'],
 ['#', '#', '#', '#', '.', '.']]

In [22]:
# Iteration 3
next_state(rules_dict, _)

[['.', '.', '#', '.', '#', '#', '.', '.', '#'],
 ['#', '.', '#', '#', '.', '.', '#', '.', '#'],
 ['#', '.', '.', '#', '#', '#', '#', '.', '.'],
 ['.', '.', '#', '#', '.', '.', '.', '#', '.'],
 ['#', '.', '#', '#', '.', '.', '#', '.', '.'],
 ['#', '.', '.', '.', '#', '.', '#', '.', '.'],
 ['.', '#', '.', '.', '#', '.', '.', '#', '#'],
 ['#', '.', '.', '#', '.', '.', '#', '.', '.'],
 ['#', '.', '.', '#', '.', '.', '.', '.', '#']]

In [23]:
# Iteration 4
next_state(rules_dict, _)

[['#', '#', '.', '#', '#', '.', '.', '#', '#', '#', '.', '#'],
 ['.', '#', '#', '.', '#', '#', '#', '.', '.', '#', '#', '.'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
 ['.', '#', '#', '#', '#', '#', '.', '#', '.', '#', '#', '#'],
 ['#', '#', '.', '#', '#', '#', '.', '.', '#', '#', '.', '.'],
 ['.', '#', '#', '.', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '.', '.', '.', '.', '.', '#', '.', '.', '.', '#', '.'],
 ['.', '#', '#', '#', '.', '.', '.', '#', '.', '.', '.', '#'],
 ['#', '#', '.', '.', '#', '#', '.', '.', '#', '.', '#', '.'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '.', '.', '#', '.'],
 ['.', '.', '#', '.', '.', '.', '#', '.', '#', '#', '#', '.'],
 ['.', '.', '.', '#', '.', '.', '.', '#', '#', '#', '#', '.']]

In [24]:
# Iteration 5
next_state(rules_dict, _)

[['.',
  '#',
  '.',
  '#',
  '.',
  '.',
  '.',
  '#',
  '.',
  '#',
  '.',
  '.',
  '.',
  '#',
  '.',
  '#',
  '.',
  '.'],
 ['#',
  '.',
  '.',
  '#',
  '.',
  '.',
  '#',
  '.',
  '.',
  '#',
  '.',
  '.',
  '#',
  '.',
  '.',
  '#',
  '.',
  '.'],
 ['#',
  '.',
  '.',
  '.',
  '#',
  '.',
  '#',
  '.',
  '.',
  '.',
  '#',
  '.',
  '#',
  '.',
  '.',
  '.',
  '#',
  '.'],
 ['#',
  '.',
  '.',
  '.',
  '.',
  '#',
  '.',
  '.',
  '#',
  '.',
  '#',
  '#',
  '#',
  '.',
  '.',
  '.',
  '.',
  '#'],
 ['#',
  '.',
  '.',
  '#',
  '.',
  '#',
  '#',
  '.',
  '#',
  '#',
  '.',
  '.',
  '#',
  '.',
  '.',
  '#',
  '.',
  '#'],
 ['.',
  '#',
  '.',
  '#',
  '.',
  '.',
  '#',
  '.',
  '.',
  '#',
  '#',
  '#',
  '.',
  '#',
  '.',
  '#',
  '.',
  '.'],
 ['.',
  '#',
  '.',
  '#',
  '.',
  '.',
  '.',
  '#',
  '#',
  '.',
  '.',
  '#',
  '.',
  '#',
  '#',
  '.',
  '.',
  '#'],
 ['#',
  '.',
  '.',
  '#',
  '.',
  '.',
  '.',
  '.',
  '.',
  '#',
  '.',
  '#',
  '.',
  '.',
  '.',
  '#',

In [25]:
finalState_ls=_

In [26]:
sum([row.count('#') for row in finalState_ls])

133